<a href="https://colab.research.google.com/github/mdonnell40/Public_Share/blob/main/NFL_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lazypredict
!pip install pytrends


In [ ]:
#turns off any warnings
import warnings
warnings.filterwarnings('ignore')

#various modules
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from lazypredict.Supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
from pytrends.request import TrendReq
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.calibration import calibration_curve

In [ ]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("3.19 Stadium")
# apiKey1 = (secret_value_0)
# secret_value_1 = user_secrets.get_secret("3.19 TeamStats")
# apiKey2 = (secret_value_1)
# secret_value_2 = user_secrets.get_secret("3.19 Betting Data")
# apiKey3 = (secret_value_2)

In [ ]:
from google.colab import userdata
# user_secrets = userdata()
secret_value_0 = userdata.get('apiKey1')
secret_value_1 = userdata.get('apiKey2')
secret_value_2 = userdata.get('apiKey3')
# userdata.get('apiKey1')
apiKey1 = (secret_value_0)
apiKey2 = (secret_value_1)
apiKey3 = (secret_value_2)

In [ ]:
#Stadium API and Staging
#API to pull in stadium details

url = "https://api.sportsdata.io/v3/nfl/scores/json/Stadiums"

#put in your own key
headers = {
    "Ocp-Apim-Subscription-Key": apiKey1
}

response = requests.get(url, headers=headers)
stadiums = response.json()

#pulling in data
df1 = pd.DataFrame(stadiums)

#dropping unneeded data
df1.drop(columns=['City', 'State', 'Country', 'Capacity', 'GeoLat', 'GeoLong'], inplace=True)

#applying numeric details for playing  surface and type
df1['PlayingSurface'] = df1['PlayingSurface'].replace({'Artificial': 1, 'Dome': 2, 'Grass': 3})
df1['Type'] = df1['Type'].replace({'Dome': 1, 'Outdoor': 2, 'RetractableDome': 3})

#renaming one of the columns
df1.rename(columns = {'Name':'Stadium'}, inplace = True)

In [ ]:
# Save df1 as a pickle file
with open('df1.pickle', 'wb') as f:
    pickle.dump(df1, f)

In [ ]:
# Load df1 from the pickle file
with open('df1.pickle', 'rb') as f:
    df1 = pickle.load(f)

In [ ]:
df1

In [ ]:
# Team Stats
import pandas as pd

# Define the JSON object
json_data = {'A': 1, 'B': 2, 'C': 3}

# Create a DataFrame with an index
teamstats = pd.DataFrame.from_dict(json_data, orient='index', columns=['Values'])

for i in range(1,19):
    url10 = f"https://api.sportsdata.io/api/nfl/odds/json/TeamGameStats/2021REG/{i}"
    url11 = f"https://api.sportsdata.io/api/nfl/odds/json/TeamGameStats/2022REG/{i}"

    headers = {
        "Ocp-Apim-Subscription-Key": apiKey2
    }

    response10 = requests.get(url10, headers=headers)
    response11 = requests.get(url11, headers=headers)

    bettingdata10 = response10.json()
    bettingdata11 = response11.json()

    e10 = pd.DataFrame.from_dict(bettingdata10, orient='index', columns=['Values'])
    e11 = pd.DataFrame.from_dict(bettingdata11, orient='index', columns=['Values'])

    teamstats = pd.concat([teamstats,e10,e11], ignore_index = True)

#bringing in regular season 2020 data
for i in range(1,18):
    url12 = f"https://api.sportsdata.io/api/nfl/odds/json/TeamGameStats/2020REG/{i}"

    headers = {
        "Ocp-Apim-Subscription-Key": apiKey2
    }

    response12 = requests.get(url12, headers=headers)

    bettingdata12 = response12.json()


    e12 = pd.DataFrame.from_dict(bettingdata12, orient='index', columns=['Values'])

    teamstats = pd.concat([teamstats,e12], ignore_index = True)

#bringing in regular season 2020, 2021 and 2022 data postseason
for i in range(1,5):
    url13 = f"https://api.sportsdata.io/api/nfl/odds/json/TeamGameStats/2021POST/{i}"
    url14 = f"https://api.sportsdata.io/api/nfl/odds/json/TeamGameStats/2022POST/{i}"
    url15 = f"https://api.sportsdata.io/api/nfl/odds/json/TeamGameStats/2020POST/{i}"

    headers = {
        "Ocp-Apim-Subscription-Key": apiKey2
    }

    response13 = requests.get(url13, headers=headers)
    response14 = requests.get(url14, headers=headers)
    response15 = requests.get(url15, headers=headers)

    bettingdata13 = response13.json()
    bettingdata14 = response14.json()
    bettingdata15 = response15.json()

    e13 = pd.DataFrame.from_dict(bettingdata13, orient='index', columns=['Values'])
    e14 = pd.DataFrame.from_dict(bettingdata14, orient='index', columns=['Values'])
    e15 = pd.DataFrame.from_dict(bettingdata15, orient='index', columns=['Values'])

    teamstats = pd.concat([teamstats,e13,e14, e15], ignore_index = True)


In [ ]:
print(teamstats.columns)

Index(['Values'], dtype='object')


In [ ]:

#dropping rows with NA's (ie bills/bengals game)
# teamstats.dropna(subset=['OpponentScore', 'TotalScore'],inplace=True)

#changing some of the stadium detail as names in the stadium section are current
# teamstats['Stadium'] = teamstats['Stadium'].replace({'Paul Brown Stadium':'Paycor Stadium','Mercedes-Benz Superdome':'Caesars Superdome','Heinz Field':'Acrisure Stadium','CenturyLink Field':'Lumen Field','Bills Stadium':'Highmark Stadium'})

#changing the date time column to date format allowing to create time based columns
# teamstats['Date'] = pd.to_datetime(teamstats['Date'])

#adding time features
# teamstats['month'] = teamstats['Date'].dt.month
# teamstats['year'] = teamstats['Date'].dt.year
# teamstats['dayofyear'] = teamstats['Date'].dt.dayofyear

In [ ]:
# Save teamstats as a pickle file
with open('teamstats.pickle', 'wb') as f:
    pickle.dump(teamstats, f)

In [ ]:
# Load teamstats from the pickle file
with open('teamstats.pickle', 'rb') as f:
    teamstats = pickle.load(f)

In [ ]:
teamstats.head()

,Values
0,1
1,2
2,3
3,401
4,Access denied due to invalid subscription key....


In [ ]:
# Five Thirty Eight
#pulling in data
df1000 = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv')

#staging data
df1000 = df1000.drop(columns=[col for col in df1000.columns if col not in ['date', 'season', 'team1', 'elo1_pre', 'elo2_pre','qbelo1_pre','qbelo2_pre','neutral']])
df1000['team1'] = df1000['team1'].replace({'OAK':'LV', 'WSH':'WAS'})
df1000.rename(columns={'date': 'Day'}, inplace=True)
df1000.rename(columns={'team1': 'HomeTeamName'}, inplace=True)

#changing the date time column to date format allowing to create time based columns
df1000['Day'] = pd.to_datetime(df1000['Day'])

#adding in date features
df1000['month'] = df1000['Day'].dt.month
df1000['year'] = df1000['Day'].dt.year
df1000['dayofyear'] = df1000['Day'].dt.dayofyear

In [ ]:
# Save df1000 as a pickle file
with open('df1000.pickle', 'wb') as f:
    pickle.dump(df1000, f)

In [ ]:
# Load df1000 from the pickle file
with open('df1000.pickle', 'rb') as f:
    df1000 = pickle.load(f)

In [ ]:
df1000.head()

,Day,season,neutral,HomeTeamName,elo1_pre,elo2_pre,qbelo1_pre,qbelo2_pre,month,year,dayofyear
0,1920-09-26,1920,0,RII,1503.95,1300.00,NaN,NaN,9,1920,270
1,1920-10-03,1920,0,AKR,1503.42,1300.00,NaN,NaN,10,1920,277
2,1920-10-03,1920,0,BFF,1478.00,1300.00,NaN,NaN,10,1920,277
3,1920-10-03,1920,0,DAY,1493.00,1504.91,NaN,NaN,10,1920,277
4,1920-10-03,1920,0,RII,1516.11,1478.00,NaN,NaN,10,1920,277


In [ ]:
#pulling in data
from google.colab import files
uploaded = files.upload()

#df2000 = pd.read_csv('https://drive.google.com/file/d/1kHI2bmRput-kmKH3dQsDsHFOyoR8DvQy')

#https://1drv.ms/u/s!AmewzQrXK2LNoH_aVWKCphF888JS?e=zFbCoz

Saving NFL_History_1.csv to NFL_History_1.csv


In [ ]:
greenline = pd.DataFrame()
greenline = pd.read_csv('NFL_History_1.csv')

In [ ]:
# Save greenline as a pickle file
with open('greenline.pickle', 'wb') as f:
    pickle.dump(greenline, f)

In [ ]:
greenline.head()

,Season,Week,Home Team,Away Team,Home Score,Away Score,Bet Type,Side,Market Line,Greenline Line,...,Cover Probability.1,Break-Even Probability.1,Difference.1,Bet Result.1,Market Line.2,Greenline Line.2,Cover Probability.2,Break-Even Probability.2,Difference.2,Bet Result.2
0,2022,1,Dallas Cowboys,Tampa Bay Buccaneers,3,19,Money Line,away,-127.00,-133.00,...,0.57,0.58,-0.02,W,-134.00,-139.00,0.58,0.57,0.01,W
1,2022,1,Dallas Cowboys,Tampa Bay Buccaneers,3,19,Money Line,home,105.00,133.00,...,0.43,0.46,-0.03,L,114.00,139.00,0.42,0.47,-0.05,L
2,2022,1,Dallas Cowboys,Tampa Bay Buccaneers,3,19,Total,under,50.50,51.60,...,0.47,0.52,-0.06,W,49.50,50.70,0.46,0.52,-0.06,W
3,2022,1,Dallas Cowboys,Tampa Bay Buccaneers,3,19,Total,over,50.50,51.60,...,0.53,0.52,0.01,L,49.50,50.70,0.54,0.52,0.01,L
4,2022,1,Dallas Cowboys,Tampa Bay Buccaneers,3,19,Spread,away,1.50,1.80,...,0.52,0.51,0.01,W,2.50,2.10,0.53,0.52,0.01,W


In [ ]:
#Five Thirty Eight
#pulling in data
df1000 = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv')

#staging data
df1000 = df1000.drop(columns=[col for col in df1000.columns if col not in ['date', 'season', 'team1', 'elo1_pre', 'elo2_pre','qbelo1_pre','qbelo2_pre','neutral']])
df1000['team1'] = df1000['team1'].replace({'OAK':'LV', 'WSH':'WAS'})
df1000.rename(columns={'date': 'Day'}, inplace=True)
df1000.rename(columns={'team1': 'HomeTeamName'}, inplace=True)

#changing the date time column to date format allowing to create time based columns
df1000['Day'] = pd.to_datetime(df1000['Day'])

#adding in date features
df1000['month'] = df1000['Day'].dt.month
df1000['year'] = df1000['Day'].dt.year
df1000['dayofyear'] = df1000['Day'].dt.dayofyear

In [ ]:
# Save df1000 as a pickle file
with open('df1000.pickle', 'wb') as f:
    pickle.dump(df1000, f)

In [ ]:
# Load df1000 from the pickle file
with open('df1000.pickle', 'rb') as f:
    df1000 = pickle.load(f)

In [ ]:
df1000.head()

,Day,season,neutral,HomeTeamName,elo1_pre,elo2_pre,qbelo1_pre,qbelo2_pre,month,year,dayofyear
0,1920-09-26,1920,0,RII,1503.95,1300.00,NaN,NaN,9,1920,270
1,1920-10-03,1920,0,AKR,1503.42,1300.00,NaN,NaN,10,1920,277
2,1920-10-03,1920,0,BFF,1478.00,1300.00,NaN,NaN,10,1920,277
3,1920-10-03,1920,0,DAY,1493.00,1504.91,NaN,NaN,10,1920,277
4,1920-10-03,1920,0,RII,1516.11,1478.00,NaN,NaN,10,1920,277


In [ ]:
#Referees
#scraping referee data
for year in [2020, 2021, 2022]:
    for ref in ['Tony-Corrente','Brad-Allen', 'Tra-Blake', 'Clete-Blakeman', 'Jerome-Boger', 'Carl-Cheffers', 'Land-Clark', 'Adrian-Hill', 'Shawn-Hochuli', 'John-Hussey', 'Alex-Kemp', 'Clay-Martin', 'Scott-Novak', 'Brad-Rogers', 'Shawn-Smith', 'Ron-Torbert', 'Bill-Vinovich', 'Craig-Wrolstad']:
        url = "https://www.nflpenalties.com/referee/" + ref + "?year=" + str(year)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

df7000 = pd.DataFrame(columns=['Date','Week','FullName','AwayTeam','Referee'])

#loop that goes through each referee games ref'd for 2020, 2021 and 2022
for ref in ['Tony-Corrente','Brad-Allen', 'Tra-Blake', 'Clete-Blakeman', 'Jerome-Boger', 'Carl-Cheffers', 'Land-Clark', 'Adrian-Hill', 'Shawn-Hochuli', 'John-Hussey', 'Alex-Kemp', 'Clay-Martin', 'Scott-Novak', 'Brad-Rogers', 'Shawn-Smith', 'Ron-Torbert', 'Bill-Vinovich', 'Craig-Wrolstad']:
    for year in [2020, 2021, 2022]:
        url = "https://www.nflpenalties.com/referee/" + ref + "?year=" + str(year)
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table',{'class':'footable'})
            rows = table.find_all('tr')

            for row in rows:
                cols = row.find_all('td')
                if len(cols) > 0:
                    date = cols[0].text.strip()
                    week = cols[1].text.strip()
                    team = cols[2].text.strip()
                    awayteam = cols[3].text.strip()
                    if date != 'Totals':
                        df7000 = df7000.append({'Date': date, 'Week': week, 'FullName': team, 'AwayTeam': awayteam, 'Referee': ref}, ignore_index=True)
        else:
            continue

#creating a data frame to expand on team name based on how data is pulled in above
df50 = pd.DataFrame({'Team': ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
                            'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'MIA', 'MIN',
                            'NE', 'NO', 'NYG', 'NYJ', 'LV', 'PHI', 'PIT', 'LAC', 'SEA', 'SF',
                            'LAR', 'TB', 'TEN', 'WAS'],
                     'FullName':['Arizona','Atlanta','Baltimore','Buffalo','Carolina','Chicago','Cincinnati','Cleveland','Dallas','Denver','Detroit','Green Bay','Houston','Indianapolis','Jacksonville','Kansas City','Miami','Minnesota','New England','New Orleans','N.Y. Giants','N.Y. Jets','Las Vegas','Philadelphia','Pittsburgh','LA Chargers','Seattle','San Francisco','LA Rams','Tampa Bay','Tennessee','Washington']})

#merging the expanded name to referee data
df7000 = pd.merge(df7000, df50[['FullName', 'Team']], on='FullName', how='left')

#staging data
df7000.rename(columns={'Date': 'Day'}, inplace=True)
df7000.rename(columns={'Team': 'HomeTeamName'}, inplace=True)

#changing ref name to a numeric value
df7000['Referee'] = df7000['Referee'].replace({'Adrian-Hill': 1, 'Alex-Kemp': 2, 'Bill-Vinovich': 3, 'Brad-Allen': 4, 'Brad-Rogers': 5, 'Carl-Cheffers': 6, 'Clay-Martin': 7, 'Clete-Blakeman': 8, 'Craig-Wrolstad': 9, 'Jerome-Boger': 10, 'John-Hussey': 11, 'Ron-Torbert': 12, 'Scott-Novak': 13,  'Bill-Vinovich': 14, 'Shawn-Hochuli': 15, 'Shawn-Smith': 16, 'Tra-Blake': 17, 'Land-Clark':18, 'Tony-Corrente': 19})

#changing the date time column to date format allowing to create time based columns
df7000['Day'] = pd.to_datetime(df7000['Day'])

#creating time based features
df7000['month'] = df7000['Day'].dt.month
df7000['year'] = df7000['Day'].dt.year
df7000['dayofyear'] = df7000['Day'].dt.dayofyear

Classes of each table:
['footable']


In [ ]:
# Save df7000 as a pickle file
with open('df7000.pickle', 'wb') as f:
    pickle.dump(df7000, f)

In [ ]:
# Load df7000 from the pickle file
with open('df7000.pickle', 'rb') as f:
    df7000 = pickle.load(f)